In [1]:
import torch
from torch import nn
import sys
import json
import os
import multiprocessing

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")
from d2lzh_pytorch.BERT_unit_v3.use_BERT import load_pretrained_model, \
SNLIBERTDataset, BERTClassifier, load_data


from d2lzh_pytorch.myUtils import try_all_gpus

In [3]:
devices = try_all_gpus()
# devices = [torch.device('cpu')]


# 加载词向量
bert, vocab = load_pretrained_model('bert.small',
                                    num_hiddens=256,
                                    ffn_num_hiddens=512,
                                    num_heads=4,
                                    num_layers=2,
                                    dropout=0.1,
                                    max_len=512,
                                    devices=devices)

In [35]:
len(vocab)

60005

In [24]:

# 加载数据
train_iter, test_iter = load_data(vocab)

read 549367 examples
read 9824 examples


In [25]:
len(train_iter), len(test_iter)

(1073, 20)

打印 train_iter 数据看看

In [33]:
# 模型及参数
net = BERTClassifier(bert)


lr = 1e-4
num_epochs = 5

trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')

In [ ]:
# 如果出现显存不足错误，请减少“batch_size”。在原始的BERT模型中，max_len=512
batch_size = 512
max_len = 128
num_workers = 4


# 开始微调
train_ch13(net,
           train_iter, 
           test_iter,
           loss, 
           trainer,
           num_epochs,
           devices)